# Toy problem for TLS solution
We want to solve here the identification problem with TLS in a very simple case: a network with 5 nodes, without any structural contraint.
However, we introduce noise in polar coordinates and we try and use our covariance matrix estimators in order to weight the noise.

In [ ]:
import mlflow
import pandas as pd
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
from scipy import sparse

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.insert(1, '..')

from src.simulation.noise import add_noise_in_polar_coordinates
from src.models.noise_transformation import naive_noise_covariance, average_true_noise_covariance
from src.models.error_in_variable import TotalLeastSquares, SparseTotalLeastSquare
from src.models.regression import ComplexLasso
from src.identification.error_metrics import fro_error, error_metrics

In [ ]:
mlflow.set_experiment('Toy problem with polar noise')

In [ ]:
samples = 300
magnitude_sd = 0.01
phase_sd = 0.01
mlflow_params = {'samples': samples, 'magnitude_sd': magnitude_sd, 'phase_sd': phase_sd}

In [ ]:
np.random.seed(11)

y_bus = np.array([
    [1+1j, 0, 0, 0, 0],
    [0, 2+1j, 0, 0, 0],
    [2+1j, 0, 1+1j, 0, 0],
    [0, 0, 0, 2+1j, 0],
    [0, 0, 0, 0, -1-1j],
])
nodes = y_bus.shape[0]

real_voltages = np.random.normal(1, 0.1, (samples, nodes)) + 1j*np.random.normal(0, 0.1, (samples, nodes))
real_currents = real_voltages @ y_bus

voltages, currents = add_noise_in_polar_coordinates(real_currents, real_voltages, magnitude_sd, phase_sd)

voltage_noise = voltages - real_voltages
current_noise = currents - real_currents

# Standard TLS

In [ ]:
mlflow.start_run(run_name='TLS')
eiv = TotalLeastSquares()
eiv.fit(voltages, currents)
y_eiv = eiv.fitted_admittance_matrix

In [ ]:
tls_metrics = error_metrics(y_bus, y_eiv)
mlflow.log_params(mlflow_params)
mlflow.log_metrics(tls_metrics.__dict__)
mlflow.end_run()
tls_metrics

# Standard LASSO

In [ ]:
mlflow.start_run(run_name='Lasso')
lasso = ComplexLasso(y_bus, verbose=False, lambdas=np.logspace(-5, 0, 50))
lasso.fit(voltages, currents)
y_lasso = lasso.fitted_admittance_matrix

In [ ]:
lasso_metrics = error_metrics(y_bus, y_lasso)
mlflow.log_param('lambda', lasso.best_trial.hyperparameters['lambda'])
mlflow.log_params(mlflow_params)
mlflow.log_metrics(lasso_metrics.__dict__)
mlflow.end_run()
lasso_metrics

In [ ]:
lasso.best_trial.hyperparameters

# L1-regularized TLS

In [ ]:
mlflow.start_run(run_name='S-TLS no cov')
tls_lambda = lasso.best_trial.hyperparameters['lambda']
max_iterations = 50
abs_tol = 10e-12
rel_tol = 10e-12
solver = cp.GUROBI
use_l1_penalty = True
use_cov_matrix = False
sparse_eiv = SparseTotalLeastSquare(lambda_value=tls_lambda, abs_tol=abs_tol, rel_tol=rel_tol, solver=solver, max_iterations=max_iterations)
sparse_eiv.fit(voltages, currents)
y_sparse_eiv = sparse_eiv.fitted_admittance_matrix

In [ ]:
stls_metrics = error_metrics(y_bus, y_sparse_eiv)
mlflow.log_param('lambda', tls_lambda)
mlflow.log_param('max_iterations', max_iterations)
mlflow.log_param('abs_tol', abs_tol)
mlflow.log_param('rel_tol', rel_tol)
mlflow.log_param('solver', solver)
mlflow.log_param('use_l1_penalty', use_l1_penalty)
mlflow.log_param('use_cov_matrix', use_cov_matrix)
mlflow.log_params(mlflow_params)
mlflow.log_metrics(stls_metrics.__dict__)
y_errors = pd.Series([fro_error(y_bus, i.fitted_parameters) for i in sparse_eiv.iterations])
targets = pd.Series([i.target_function for i in sparse_eiv.iterations])
for i in range(len(y_errors)):
    mlflow.log_metric('fro_error_evo', value=y_errors[i], step=i)
    mlflow.log_metric('opt_cost_evo', value=targets[i], step=i)
mlflow.end_run()
stls_metrics

In [ ]:
y_errors.plot(title='Fro error on Y');

In [ ]:
targets.plot(title='Target function');

# L1-regularized TLS with estimated covariance matrix

In [ ]:
sigma_voltage = average_true_noise_covariance(voltages, magnitude_sd, phase_sd)
sigma_current = average_true_noise_covariance(currents, magnitude_sd, phase_sd)

inv_sigma_current = sparse.linalg.inv(sigma_current)
inv_sigma_voltage = sparse.linalg.inv(sigma_voltage)

In [ ]:
mlflow.start_run(run_name='S-TLS')
use_cov_matrix = True
sparse_eiv_cov = SparseTotalLeastSquare(lambda_value=tls_lambda, abs_tol=abs_tol, rel_tol=rel_tol, solver=solver, max_iterations=max_iterations)
sparse_eiv_cov.fit(voltages, currents, inv_sigma_voltage, inv_sigma_current)
y_sparse_eiv_cov = sparse_eiv_cov.fitted_admittance_matrix

In [ ]:
stls_cov_metrics = error_metrics(y_bus, y_sparse_eiv_cov)
mlflow.log_param('lambda', tls_lambda)
mlflow.log_param('max_iterations', max_iterations)
mlflow.log_param('abs_tol', abs_tol)
mlflow.log_param('rel_tol', rel_tol)
mlflow.log_param('solver', solver)
mlflow.log_param('use_l1_penalty', use_l1_penalty)
mlflow.log_param('use_cov_matrix', use_cov_matrix)
mlflow.log_params(mlflow_params)
mlflow.log_metrics(stls_cov_metrics.__dict__)
y_errors = pd.Series([fro_error(y_bus, i.fitted_parameters) for i in sparse_eiv_cov.iterations])
targets = pd.Series([i.target_function for i in sparse_eiv_cov.iterations])
for i in range(len(y_errors)):
    mlflow.log_metric('fro_error_evo', value=y_errors[i], step=i)
    mlflow.log_metric('opt_cost_evo', value=targets[i], step=i)
mlflow.end_run()
stls_cov_metrics

In [ ]:
y_errors.plot(title='Fro error on Y');

In [ ]:
targets.plot(title='Target function');